In [1]:
import sys 
import os 
import numpy as np
import pandas as pd
import sklearn as sk
import re
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE
from sklearn import svm
from sklearn import metrics as metrics_sk
from sklearn.metrics import precision_recall_curve
from scipy.stats import ks_2samp
from IPython.display import Image, display_svg, SVG

In [2]:
from keras import layers, Sequential
from keras.layers import Input, Add, Dropout, Dense, Activation, ZeroPadding1D, ZeroPadding2D, BatchNormalization, Flatten, Conv1D, AveragePooling1D, MaxPooling1D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras import metrics
from keras import optimizers
from keras import regularizers
import pydot
from tensorflow.keras import regularizers
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline
import tensorflow as tf
import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

Using TensorFlow backend.
C:\Anaconda3\envs\benedito_deep\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Anaconda3\envs\benedito_deep\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Anaconda3\envs\benedito_deep\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Anaconda3\envs\benedito_deep\lib\site-packages\tensorflo

In [3]:
from keras.backend import sigmoid
def swish(x, beta = 1):
    return (x * sigmoid(beta * x))

In [4]:
from keras.backend import log
def inv_h_sin(x, theta = 1):
    if x == 0:
        return x
    else:
        return log(theta*x + ((theta*x)**2 + 1)**0.5)/theta

In [5]:
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
get_custom_objects().update({'swish': Activation(swish)})
get_custom_objects().update({'inv_h_sin': Activation(inv_h_sin)})

In [ ]:
os.getcwd()

In [6]:
pd.set_option('display.max_columns', None)

In [7]:
os.chdir("C:/Users/Benedito/Desktop/MECAI/Introdução a Estatistica/MVP")

In [8]:
os.getcwd()

'C:\\Users\\Benedito\\Desktop\\MECAI\\Introdução a Estatistica\\MVP'

In [9]:
df = pd.read_csv("modelling_table5.csv")

C:\Anaconda3\envs\benedito_deep\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (623,625,626,629) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:
data = df

In [11]:
data.head()

,Unnamed: 0,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_other,REGION_RATING_CLIENT_2,REGION_RATING_CLIENT_3,REGION_RATING_CLIENT_W_CITY_2,REGION_RATING_CLIENT_W_CITY_3,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_other,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes,cnt_appl_previous,cnt_revolving_previous,cnt_cash_previous,cnt_consumer_previous,avg_amt_annuity_previous,avg_amt_application_previous,avg_amt_credit_previous,avg_amt_down_payment_previous,avg_amt_goods_price_previous,cnt_appl_start_bus_day_previous,cnt_appl_start_weekend_previous,cnt_appl_s

In [ ]:
data.dtypes.to_csv("modelling_5_dtypes.csv")

In [12]:
categorical_feats = ['MODE(previous.CHANNEL_TYPE)',
'MODE(previous.CODE_REJECT_REASON)',
'MODE(previous.FLAG_LAST_APPL_PER_CONTRACT)',
'MODE(previous.NAME_CASH_LOAN_PURPOSE)',
'MODE(previous.NAME_CLIENT_TYPE)',
'MODE(previous.NAME_CONTRACT_STATUS)',
'MODE(previous.NAME_CONTRACT_TYPE)',
'MODE(previous.NAME_GOODS_CATEGORY)',
'MODE(previous.NAME_PAYMENT_TYPE)',
'MODE(previous.NAME_PORTFOLIO)',
'MODE(previous.NAME_PRODUCT_TYPE)',
'MODE(previous.NAME_SELLER_INDUSTRY)',
'MODE(previous.NAME_TYPE_SUITE)',
'MODE(previous.NAME_YIELD_GROUP)',
'MODE(previous.PRODUCT_COMBINATION)',
'MODE(previous.WEEKDAY_APPR_PROCESS_START)',
'N_MOST_COMMON(previous.CHANNEL_TYPE)[0]',
'N_MOST_COMMON(previous.CHANNEL_TYPE)[1]',
'N_MOST_COMMON(previous.CHANNEL_TYPE)[2]',
'N_MOST_COMMON(previous.CODE_REJECT_REASON)[0]',
'N_MOST_COMMON(previous.CODE_REJECT_REASON)[1]',
'N_MOST_COMMON(previous.CODE_REJECT_REASON)[2]',
'N_MOST_COMMON(previous.FLAG_LAST_APPL_PER_CONTRACT)[0]',
'N_MOST_COMMON(previous.FLAG_LAST_APPL_PER_CONTRACT)[1]',
'N_MOST_COMMON(previous.NAME_CASH_LOAN_PURPOSE)[0]',
'N_MOST_COMMON(previous.NAME_CASH_LOAN_PURPOSE)[1]',
'N_MOST_COMMON(previous.NAME_CASH_LOAN_PURPOSE)[2]',
'N_MOST_COMMON(previous.NAME_CLIENT_TYPE)[0]',
'N_MOST_COMMON(previous.NAME_CLIENT_TYPE)[1]',
'N_MOST_COMMON(previous.NAME_CLIENT_TYPE)[2]',
'N_MOST_COMMON(previous.NAME_CONTRACT_STATUS)[0]',
'N_MOST_COMMON(previous.NAME_CONTRACT_STATUS)[1]',
'N_MOST_COMMON(previous.NAME_CONTRACT_STATUS)[2]',
'N_MOST_COMMON(previous.NAME_CONTRACT_TYPE)[0]',
'N_MOST_COMMON(previous.NAME_CONTRACT_TYPE)[1]',
'N_MOST_COMMON(previous.NAME_CONTRACT_TYPE)[2]',
'N_MOST_COMMON(previous.NAME_GOODS_CATEGORY)[0]',
'N_MOST_COMMON(previous.NAME_GOODS_CATEGORY)[1]',
'N_MOST_COMMON(previous.NAME_GOODS_CATEGORY)[2]',
'N_MOST_COMMON(previous.NAME_PAYMENT_TYPE)[0]',
'N_MOST_COMMON(previous.NAME_PAYMENT_TYPE)[1]',
'N_MOST_COMMON(previous.NAME_PAYMENT_TYPE)[2]',
'N_MOST_COMMON(previous.NAME_PORTFOLIO)[0]',
'N_MOST_COMMON(previous.NAME_PORTFOLIO)[1]',
'N_MOST_COMMON(previous.NAME_PORTFOLIO)[2]',
'N_MOST_COMMON(previous.NAME_PRODUCT_TYPE)[0]',
'N_MOST_COMMON(previous.NAME_PRODUCT_TYPE)[1]',
'N_MOST_COMMON(previous.NAME_PRODUCT_TYPE)[2]',
'N_MOST_COMMON(previous.NAME_SELLER_INDUSTRY)[0]',
'N_MOST_COMMON(previous.NAME_SELLER_INDUSTRY)[1]',
'N_MOST_COMMON(previous.NAME_SELLER_INDUSTRY)[2]',
'N_MOST_COMMON(previous.NAME_TYPE_SUITE)[0]',
'N_MOST_COMMON(previous.NAME_TYPE_SUITE)[1]',
'N_MOST_COMMON(previous.NAME_TYPE_SUITE)[2]',
'N_MOST_COMMON(previous.NAME_YIELD_GROUP)[0]',
'N_MOST_COMMON(previous.NAME_YIELD_GROUP)[1]',
'N_MOST_COMMON(previous.NAME_YIELD_GROUP)[2]',
'N_MOST_COMMON(previous.PRODUCT_COMBINATION)[0]',
'N_MOST_COMMON(previous.PRODUCT_COMBINATION)[1]',
'N_MOST_COMMON(previous.PRODUCT_COMBINATION)[2]',
'N_MOST_COMMON(previous.WEEKDAY_APPR_PROCESS_START)[0]',
'N_MOST_COMMON(previous.WEEKDAY_APPR_PROCESS_START)[1]',
'N_MOST_COMMON(previous.WEEKDAY_APPR_PROCESS_START)[2]',
'MODE(bureau.CREDIT_ACTIVE)',
'MODE(bureau.CREDIT_CURRENCY)',
'MODE(bureau.CREDIT_TYPE)',
'N_MOST_COMMON(bureau.CREDIT_ACTIVE)[0]',
'N_MOST_COMMON(bureau.CREDIT_ACTIVE)[1]',
'N_MOST_COMMON(bureau.CREDIT_ACTIVE)[2]',
'N_MOST_COMMON(bureau.CREDIT_CURRENCY)[0]',
'N_MOST_COMMON(bureau.CREDIT_CURRENCY)[1]',
'N_MOST_COMMON(bureau.CREDIT_CURRENCY)[2]',
'N_MOST_COMMON(bureau.CREDIT_TYPE)[0]',
'N_MOST_COMMON(bureau.CREDIT_TYPE)[1]',
'N_MOST_COMMON(bureau.CREDIT_TYPE)[2]']

In [13]:
data.head()

,Unnamed: 0,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_other,REGION_RATING_CLIENT_2,REGION_RATING_CLIENT_3,REGION_RATING_CLIENT_W_CITY_2,REGION_RATING_CLIENT_W_CITY_3,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_other,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes,cnt_appl_previous,cnt_revolving_previous,cnt_cash_previous,cnt_consumer_previous,avg_amt_annuity_previous,avg_amt_application_previous,avg_amt_credit_previous,avg_amt_down_payment_previous,avg_amt_goods_price_previous,cnt_appl_start_bus_day_previous,cnt_appl_start_weekend_previous,cnt_appl_s

In [14]:
data[categorical_feats].fillna("no_info", inplace = True)

C:\Anaconda3\envs\benedito_deep\lib\site-packages\pandas\core\frame.py:4323: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [15]:
data.head()

,Unnamed: 0,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_other,REGION_RATING_CLIENT_2,REGION_RATING_CLIENT_3,REGION_RATING_CLIENT_W_CITY_2,REGION_RATING_CLIENT_W_CITY_3,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_other,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes,cnt_appl_previous,cnt_revolving_previous,cnt_cash_previous,cnt_consumer_previous,avg_amt_annuity_previous,avg_amt_application_previous,avg_amt_credit_previous,avg_amt_down_payment_previous,avg_amt_goods_price_previous,cnt_appl_start_bus_day_previous,cnt_appl_start_weekend_previous,cnt_appl_s

In [16]:
def check_string(x):
    if x in ('NaN', 'NA', 'n/a', 'N/A', 'inf', '-inf', 'Inf', '-Inf'):
        return True
    else:
        return False

In [17]:
def fill_cat(df, lista_cat):
    for cat_feat in lista_cat:
        df.loc[pd.isna(df[cat_feat]), cat_feat] = 'no_info'
        df.loc[df[cat_feat].isnull(), cat_feat] = 'no_info'
        df.loc[df[cat_feat].apply(check_string), cat_feat] = 'no_info'
    return df
    

In [18]:
data = fill_cat(data, categorical_feats)

In [19]:
data.fillna(0, inplace = True)

In [20]:
data.head()

,Unnamed: 0,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_other,REGION_RATING_CLIENT_2,REGION_RATING_CLIENT_3,REGION_RATING_CLIENT_W_CITY_2,REGION_RATING_CLIENT_W_CITY_3,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_other,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes,cnt_appl_previous,cnt_revolving_previous,cnt_cash_previous,cnt_consumer_previous,avg_amt_annuity_previous,avg_amt_application_previous,avg_amt_credit_previous,avg_amt_down_payment_previous,avg_amt_goods_price_previous,cnt_appl_start_bus_day_previous,cnt_appl_start_weekend_previous,cnt_appl_s

In [21]:
data = data.drop(['Unnamed: 0'], axis = 1)

In [22]:
data = data.drop(categorical_feats, axis = 1)

In [23]:
data.set_index('SK_ID_CURR', inplace=True)

In [24]:
data = data.replace([np.inf, -np.inf], 0)

In [25]:
data.head()

,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_other,REGION_RATING_CLIENT_2,REGION_RATING_CLIENT_3,REGION_RATING_CLIENT_W_CITY_2,REGION_RATING_CLIENT_W_CITY_3,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_other,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Yes,cnt_appl_previous,cnt_revolving_previous,cnt_cash_previous,cnt_consumer_previous,avg_amt_annuity_previous,avg_amt_application_previous,avg_amt_credit_previous,avg_amt_down_payment_previous,avg_amt_goods_price_previous,cnt_appl_start_bus_day_previous,cnt_appl_start_weekend_previous,cnt_appl_start_dawn_previous,cnt

In [ ]:
#data = pd.get_dummies(data, columns = categorical_feats, drop_first = True)

In [26]:
feat_selected = ["mean_knn_target",
"EXT_SOURCE_3",
"EXT_SOURCE_2",
"EXT_SOURCE_1",
"SUM(previous.CNT_PAYMENT)",
"amt_credit_sum_consumer_credit_bureau",
"avg_pos_cash_cnt_installment_future_previous",
"CODE_GENDER_M",
"amt_credit_sum_debt_credit_card_bureau",
"cnt_refused_previous",
"NAME_CONTRACT_TYPE_Revolving loans",
"max_pos_cash_sk_dpd_def_previous",
"SUM(previous.DAYS_FIRST_DRAWING)",
"amt_paid_down",
"NAME_EDUCATION_TYPE_Higher education",
"STD(previous.DAYS_FIRST_DRAWING)",
"NAME_FAMILY_STATUS_Married",
"MIN(previous.CNT_PAYMENT)",
"cnt_yield_low_normal_previous",
"avg_amt_application_previous",
"credit_income_ratio",
"remaining_pmts",
"OCCUPATION_TYPE_Sales staff",
"MAX(previous.NFLAG_LAST_APPL_IN_DAY)",
"FLAG_EMAIL",
"missing_rate",
"REGION_RATING_CLIENT_W_CITY_2",
"COUNT(bureau)",
"gap_avg_application_credit_previous",
"days_phone_change_over_gap_amt_installment",
"FLAG_OWN_CAR_Y",
"avg_pos_cash_cnt_installment_previous",
"NAME_EDUCATION_TYPE_Secondary / secondary special",
"YEARS_BUILD_MODE",
"UI_credit_card_bureau",
"ENTRANCES_AVG",
"num_loans_l1m",
"STD(bureau.CREDIT_DAY_OVERDUE)",
"STD(previous.AMT_GOODS_PRICE)",
"max_interest_rate_privileged_previous",
"cnt_contracts_bureau",
"SKEW(previous.DAYS_TERMINATION)",
"MEAN(previous.DAYS_LAST_DUE_1ST_VERSION)",
"MIN(previous.AMT_APPLICATION)",
"MIN(bureau.DAYS_CREDIT)",
"delinquency_6_mths_consumer_credit_bureau",
"MAX(previous.DAYS_FIRST_DRAWING)",
"NAME_EDUCATION_TYPE_Lower secondary",
"MIN(previous.HOUR_APPR_PROCESS_START)",
"avg_amt_credit_card_receivable_principal_previous",
"MEAN(bureau.AMT_CREDIT_SUM)",
"amt_credit_max_overdue_bureau",
"DAYS_ID_PUBLISH",
"tot_debt_lim_ratio",
"avg_amt_credit_card_total_receivable_previous",
"max_interest_rate_primary_previous",
"MEDIAN(previous.NFLAG_INSURED_ON_APPROVAL)",
"num_loans_l6m",
"STD(bureau.DAYS_CREDIT)",
"MEAN(previous.RATE_INTEREST_PRIMARY)",
"FLOORSMIN_AVG",
"SUM(bureau.AMT_CREDIT_SUM_DEBT)",
"avg_cnt_credit_card_drawings_atm_previous",
"APARTMENTS_MEDI",
"LIVINGAREA_MODE",
"delinquency_12_24_mths_microloan_bureau",
"SKEW(bureau.DAYS_CREDIT)",
"LANDAREA_AVG",
"num_loans_l3m",
"FLAG_DOCUMENT_13",
"MAX(bureau.AMT_CREDIT_SUM_DEBT)",
"MAX(previous.CNT_PAYMENT)",
"delinquency_6_12_mths_credit_card_bureau",
"MAX(bureau.AMT_CREDIT_SUM_LIMIT)",
"MEAN(previous.AMT_CREDIT)",
"MAX(bureau.DAYS_CREDIT_UPDATE)",
"WALLSMATERIAL_MODE_Monolithic",
"avg_amt_credit_card_drawings_atm_previous",
"MEDIAN(previous.CNT_PAYMENT)",
"avg_cnt_credit_card_instalment_mature_cum_previous",
"FLAG_PHONE",
"YEARS_BEGINEXPLUATATION_AVG",
"SUM(previous.DAYS_TERMINATION)",
"income_over_credit",
"days_phone_change_times_gap_days_installment",
"SUM(previous.AMT_GOODS_PRICE)",
"LANDAREA_MEDI",
"max_cnt_payment_previous",
"SUM(bureau.AMT_CREDIT_SUM_OVERDUE)",
"MEAN(previous.NFLAG_LAST_APPL_IN_DAY)",
"MIN(previous.RATE_DOWN_PAYMENT)",
"SKEW(bureau.AMT_CREDIT_SUM_LIMIT)",
"SUM(previous.RATE_DOWN_PAYMENT)",
"cnt_portfolio_cash_previous",
"MEDIAN(bureau.CNT_CREDIT_PROLONG)",
"SKEW(previous.DAYS_DECISION)",
"NAME_INCOME_TYPE_State servant",
"STD(previous.AMT_CREDIT)",
"MEAN(bureau.DAYS_ENDDATE_FACT)",
"NAME_TYPE_SUITE_Other_A",
"FLOORSMIN_MEDI",
"DEF_30_CNT_SOCIAL_CIRCLE",
"REG_REGION_NOT_WORK_REGION",
"SKEW(bureau.AMT_CREDIT_MAX_OVERDUE)",
"NONLIVINGAREA_MEDI",
"delinquency_6_12_mths_microloan_bureau",
"avg_amt_credit_card_recivable_previous",
"delinquency_long_total_bureau",
"FLAG_WORK_PHONE",
"cnt_channel_contact_center_previous",
"SUM(previous.DAYS_FIRST_DUE)",
"OWN_CAR_AGE",
"AMT_GOODS_PRICE",
"cnt_appl_previous",
"delinquency_12_24_mths_consumer_credit_bureau",
"cnt_cash_previous",
"NAME_TYPE_SUITE_Family",
"MEAN(previous.AMT_DOWN_PAYMENT)",
"FLAG_DOCUMENT_3",
"NAME_TYPE_SUITE_Group of people",
"cnt_seller_furniture_previous",
"delinquency_6_12_mths_total_bureau",
"CODE_GENDER_XNA",
"amt_credit_sum_working_capital_bureau",
"MEDIAN(previous.RATE_DOWN_PAYMENT)",
"DAYS_REGISTRATION",
"DEF_60_CNT_SOCIAL_CIRCLE",
"SUM(bureau.DAYS_CREDIT_ENDDATE)",
"MIN(bureau.DAYS_CREDIT_ENDDATE)",
"delinquency_24_36_mths_consumer_credit_bureau",
"MAX(previous.AMT_DOWN_PAYMENT)",
"UI_credit_sum_total_bureau",
"SKEW(bureau.DAYS_ENDDATE_FACT)",
"SKEW(previous.AMT_DOWN_PAYMENT)",
"NAME_INCOME_TYPE_Student",
"MEDIAN(bureau.DAYS_CREDIT_UPDATE)",
"credit_income_cluster_4",
"YEARS_BEGINEXPLUATATION_MEDI",
"NAME_HOUSING_TYPE_Office apartment",
"max_credit_card_sk_dpd_def_previous",
"SKEW(previous.AMT_APPLICATION)",
"cnt_product_xsell_previous",
"WALLSMATERIAL_MODE_Stone, brick",
"cnt_yield_low_action_previous",
"MAX(previous.DAYS_DECISION)",
"amt_credit_sum_overdue_microloan_bureau",
"MEAN(previous.DAYS_TERMINATION)",
"OBS_30_CNT_SOCIAL_CIRCLE",
"cnt_seller_electronics_previous",
"MAX(previous.HOUR_APPR_PROCESS_START)",
"WALLSMATERIAL_MODE_Panel",
"MEDIAN(previous.AMT_APPLICATION)",
"down_payment_times_cnt_refused_previous",
"MEDIAN(previous.AMT_CREDIT)",
"max_installments_gap_amt_payment_previous",
"STD(previous.CNT_PAYMENT)",
"FLOORSMIN_MODE",
"FLOORSMAX_AVG",
"STD(bureau.DAYS_ENDDATE_FACT)"]

In [ ]:
data.to_csv("modelling_table_6.csv")

In [27]:
np.random.seed(11235)
sample = data.sample(200000, random_state = 11235)

In [28]:
valid = data[~data.index.isin(sample.index)].sample(100000, random_state = 11235)
test_final = data[~((data.index.isin(valid.index))|data.index.isin(sample.index))]

In [29]:
X = sample.drop(['TARGET'], axis =1)
y = sample['TARGET']

In [30]:
X = X[feat_selected]

In [31]:
X.fillna(0, inplace = True)

In [ ]:
#smote = SMOTE(sampling_strategy = 0.1, random_state = 11235) ##0.15 fica bom

In [ ]:
#X_, y_ = X, y

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.3, random_state=1234)

In [ ]:
#from keras.callbacks import EarlyStopping

In [ ]:
#callback = EarlyStopping(monitor='AUC', patience=5)

## Neural Network

In [32]:
### MELHOR DE TODAS
NN_Model = Sequential()
NN_Model.add(Dense(32, input_dim=X.shape[-1], activation='swish', kernel_initializer = 'random_normal', bias_initializer = 'zeros'))
NN_Model.add(Dropout(0.5,seed = 11235))
NN_Model.add(Dense(256, activation='swish', kernel_initializer = 'random_normal', bias_initializer = 'zeros'))
NN_Model.add(Dropout(0.5,seed = 11235))
NN_Model.add(Dense(32, activation='swish', kernel_initializer = 'random_normal', bias_initializer = 'zeros'))
NN_Model.add(Dropout(0.5, seed = 11235))
NN_Model.add(Dense(1, activation='sigmoid', kernel_initializer = 'random_normal', bias_initializer = 'zeros'))

In [33]:
metrics_ = [metrics.FalseNegatives(name="fn"),metrics.FalsePositives(name="fp"),
           metrics.TrueNegatives(name="tn"),
           metrics.TruePositives(name="tp"),
           metrics.Precision(name="precision"),
           metrics.Recall(name="recall"),
           metrics.AUC(name="AUC"), 
           metrics.Accuracy(name="accuracy")]

In [ ]:
#opt = optimizers.Nadam()

In [34]:
NN_Model.compile(loss='binary_crossentropy', optimizer="adam", metrics=metrics_)
#tf.keras.metrics.Precision(), tf.keras.metrics.Recall() tf.keras.metrics.AUC()

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [35]:
class_weight = {0: 1/(y==0).sum(), 1: 1/(y==1).sum()}

In [36]:
scaler = StandardScaler()

In [37]:
feats = X.columns
X = scaler.fit_transform(X)

X_valid = scaler.transform(valid[feats])
y_valid = valid['TARGET']
X_test_final = scaler.transform(test_final[feats])
y_test_final = test_final['TARGET']

In [38]:
NN_Model.fit(x = X, y = y, epochs = 128, batch_size = 2048, class_weight = class_weight)#, callbacks = [callback])


Epoch 1/128
200000/200000 [==============================] - 11s 57us/step - loss: 6.9325e-06 - fn: 8064.0000 - fp: 92521.0000 - tn: 91321.0000 - tp: 8094.0000 - precision: 0.0804 - recall: 0.5009 - AUC: 0.4989 - accuracy: 0.0000e+00
Epoch 2/128
200000/200000 [==============================] - 4s 18us/step - loss: 6.9300e-06 - fn: 8301.0000 - fp: 84610.0000 - tn: 99232.0000 - tp: 7857.0000 - precision: 0.0850 - recall: 0.4863 - AUC: 0.5146 - accuracy: 0.0000e+00
Epoch 3/128
200000/200000 [==============================] - 4s 18us/step - loss: 6.9277e-06 - fn: 8060.0000 - fp: 83385.0000 - tn: 100457.0000 - tp: 8098.0000 - precision: 0.0885 - recall: 0.5012 - AUC: 0.5220 - accuracy: 0.0000e+00
Epoch 4/128
200000/200000 [==============================] - 4s 19us/step - loss: 6.9239e-06 - fn: 7620.0000 - fp: 83787.0000 - tn: 100055.0000 - tp: 8538.0000 - precision: 0.0925 - recall: 0.5284 - AUC: 0.5424 - accuracy: 0.0000e+00
Epoch 5/128
200000/200000 [==============================] - 4s 

200000/200000 [==============================] - 4s 18us/step - loss: 5.7784e-06 - fn: 4836.0000 - fp: 52309.0000 - tn: 131533.0000 - tp: 11322.0000 - precision: 0.1779 - recall: 0.7007 - AUC: 0.7705 - accuracy: 5.0000e-06
Epoch 34/128
200000/200000 [==============================] - 4s 18us/step - loss: 5.7713e-06 - fn: 4877.0000 - fp: 52236.0000 - tn: 131606.0000 - tp: 11281.0000 - precision: 0.1776 - recall: 0.6982 - AUC: 0.7714 - accuracy: 0.0000e+00
Epoch 35/128
200000/200000 [==============================] - 4s 18us/step - loss: 5.7697e-06 - fn: 4851.0000 - fp: 52033.0000 - tn: 131809.0000 - tp: 11307.0000 - precision: 0.1785 - recall: 0.6998 - AUC: 0.7712 - accuracy: 1.0000e-05
Epoch 36/128
200000/200000 [==============================] - 4s 18us/step - loss: 5.7631e-06 - fn: 4859.0000 - fp: 51914.0000 - tn: 131928.0000 - tp: 11299.0000 - precision: 0.1787 - recall: 0.6993 - AUC: 0.7716 - accuracy: 5.0000e-06 - loss: 5.7350e-06 - fn: 1039.0000 - fp: 11105.0000 - tn: 28438.0000 

200000/200000 [==============================] - 4s 20us/step - loss: 5.7002e-06 - fn: 4776.0000 - fp: 52229.0000 - tn: 131613.0000 - tp: 11382.0000 - precision: 0.1789 - recall: 0.7044 - AUC: 0.7766 - accuracy: 5.0000e-06 loss: 5.5913e-06 - fn: 241.0000 - fp: 2646.0000 - tn: 6807.0000 - tp: 546.0000 - precision: 0.171
Epoch 66/128
200000/200000 [==============================] - 4s 19us/step - loss: 5.6840e-06 - fn: 4723.0000 - fp: 52113.0000 - tn: 131729.0000 - tp: 11435.0000 - precision: 0.1799 - recall: 0.7077 - AUC: 0.7781 - accuracy: 1.0000e-05
Epoch 67/128
200000/200000 [==============================] - 4s 18us/step - loss: 5.6837e-06 - fn: 4725.0000 - fp: 52256.0000 - tn: 131586.0000 - tp: 11433.0000 - precision: 0.1795 - recall: 0.7076 - AUC: 0.7780 - accuracy: 0.0000e+00
Epoch 68/128
200000/200000 [==============================] - 4s 18us/step - loss: 5.6858e-06 - fn: 4730.0000 - fp: 52245.0000 - tn: 131597.0000 - tp: 11428.0000 - precision: 0.1795 - recall: 0.7073 - AUC: 0

200000/200000 [==============================] - 4s 18us/step - loss: 5.6643e-06 - fn: 4716.0000 - fp: 51703.0000 - tn: 132139.0000 - tp: 11442.0000 - precision: 0.1812 - recall: 0.7081 - AUC: 0.7801 - accuracy: 1.5000e-05
Epoch 99/128
200000/200000 [==============================] - 4s 18us/step - loss: 5.6543e-06 - fn: 4684.0000 - fp: 52505.0000 - tn: 131337.0000 - tp: 11474.0000 - precision: 0.1793 - recall: 0.7101 - AUC: 0.7804 - accuracy: 5.0000e-06
Epoch 100/128
200000/200000 [==============================] - 4s 18us/step - loss: 5.6581e-06 - fn: 4740.0000 - fp: 51989.0000 - tn: 131853.0000 - tp: 11418.0000 - precision: 0.1801 - recall: 0.7066 - AUC: 0.7799 - accuracy: 5.0000e-06
Epoch 101/128
200000/200000 [==============================] - 4s 18us/step - loss: 5.6608e-06 - fn: 4661.0000 - fp: 52477.0000 - tn: 131365.0000 - tp: 11497.0000 - precision: 0.1797 - recall: 0.7115 - AUC: 0.7802 - accuracy: 1.0000e-05
Epoch 102/128
200000/200000 [==============================] - 4s 1

In [39]:
predictions = NN_Model.predict(X_valid)
predictions_ = (predictions > 0.5).astype(int)

In [40]:
acc_valid = metrics_sk.accuracy_score(y_valid, (NN_Model.predict(X_valid)>0.5))
f1_valid = metrics_sk.f1_score(y_valid, (NN_Model.predict(X_valid)>0.5))
precision_valid = metrics_sk.precision_score(y_valid, (NN_Model.predict(X_valid)>0.5))
recall_valid = metrics_sk.recall_score(y_valid, (NN_Model.predict(X_valid)>0.5))
auc_valid = metrics_sk.roc_auc_score(y_valid, NN_Model.predict(X_valid))

print('accuracy_valid: ',acc_valid)
print('precision_valid: ',precision_valid)
print('recall_valid: ',recall_valid)
print('F1_valid: ',f1_valid)
print('AUC_valid: ',auc_valid)

accuracy_valid:  0.7154
precision_valid:  0.17822372966522204
recall_valid:  0.6994420334779913
F1_valid:  0.2837164847326324
AUC_valid:  0.7744943209863057


## LightGBM

In [ ]:
#dtree = LGBMClassifier(max_depth = 2, min_samples_split=100, min_data_in_leaf = 250,n_estimators=300,
#                            min_impurity_decrease=1e-03, lambda_l1=0.3,
#                       lambda_l2=0.7, random_state=11235, eta = 0.005)

In [ ]:
###GRID SEARCH LGBM ---> NAO RODAR SEMPRE

list_i = [2, 4, 6]
list_j = [50, 100, 200]
list_k = [200, 300, 400]
list_l = [100, 200, 300]

for i in list_i:
    for j in list_j:
        for k in list_k:
            for l in list_l:
                dtree = LGBMClassifier(max_depth = i, min_samples_split=j, min_data_in_leaf = k,n_estimators=l,
                            min_impurity_decrease=1e-03, lambda_l1=0.3, lambda_l2=0.7, random_state=11235, eta = 0.005)
                class_tree = dtree.fit(X_train, y_train)
                auc_valid = metrics_sk.roc_auc_score(y_valid, class_tree.predict_proba(X_valid)[:,1])
                print('depth: ', i, 'samples_split: ', j, 'data_in_leaf: ', k, 'n_estimators: ', l, 'auc_valid: ', auc_valid)

In [41]:
dtree = LGBMClassifier(max_depth = 4, min_samples_split=50, min_data_in_leaf = 400,n_estimators=300,
                            min_impurity_decrease=1e-03, lambda_l1=0.3,
                       lambda_l2=0.7, random_state=11235, eta = 0.005)

In [42]:
class_tree = dtree.fit(X, y)

In [43]:
acc_valid = metrics_sk.accuracy_score(y_valid, class_tree.predict(X_valid))
f1_valid = metrics_sk.f1_score(y_valid, class_tree.predict(X_valid))
precision_valid = metrics_sk.precision_score(y_valid, class_tree.predict(X_valid))
recall_valid = metrics_sk.recall_score(y_valid, class_tree.predict(X_valid))
auc_valid = metrics_sk.roc_auc_score(y_valid, class_tree.predict_proba(X_valid)[:,1])

print('accuracy_valid: ',acc_valid)
print('precision_valid: ',precision_valid)
print('recall_valid: ',recall_valid)
print('F1_valid: ',f1_valid)
print('AUC_valid: ',auc_valid)

accuracy_valid:  0.92017
precision_valid:  0.5737410071942446
recall_valid:  0.039553626782393055
F1_valid:  0.07400533580791091
AUC_valid:  0.7864300659064931


## Stacking NN+LGBM

In [44]:
pred_nn = NN_Model.predict(X_valid)
pred_lgbm = class_tree.predict_proba(X_valid)[:,1].reshape(-1, 1)


In [45]:
X_stack = np.zeros((pred_nn.shape[0], 2))

In [46]:
X_stack[:,0] = pred_nn.reshape(1, -1)
X_stack[:,1] = pred_lgbm.reshape(1, -1)
#X_stack[:,2] = pred_nn.reshape(1, -1)**2
#X_stack[:,3] = pred_lgbm.reshape(1, -1)**2
#X_stack[:,4] = pred_nn.reshape(1, -1)**3
#X_stack[:,5] = pred_lgbm.reshape(1, -1)**3

In [47]:
class_weight = {0: 1/(y_valid==0).sum(), 1: 1/(y_valid==1).sum()}

In [48]:
lr = LogisticRegression()#class_weight=class_weight

In [49]:
from sklearn.naive_bayes import GaussianNB

In [50]:
nb = GaussianNB(var_smoothing = 0.01)

In [51]:
lr_ = lr.fit(X_stack, y_valid)

In [52]:
nb_ = nb.fit(X_stack, y_valid)

## Validação - supervisor logistico

In [53]:
acc_stack = metrics_sk.accuracy_score(y_valid, lr_.predict(X_stack))
f1_stack = metrics_sk.f1_score(y_valid, lr_.predict(X_stack))
precision_stack = metrics_sk.precision_score(y_valid, lr_.predict(X_stack))
recall_stack = metrics_sk.recall_score(y_valid, lr_.predict(X_stack))
auc_stack = metrics_sk.roc_auc_score(y_valid, lr_.predict_proba(X_stack)[:,1])

print('accuracy_stack: ',acc_stack)
print('precision_stack: ',precision_stack)
print('recall_stack: ',recall_stack)
print('F1_stack: ',f1_stack)
print('AUC_stack: ',auc_stack)

accuracy_stack:  0.92002
precision_stack:  0.5405078597339782
recall_stack:  0.05542467451952883
F1_stack:  0.10053981106612686
AUC_stack:  0.7828901123603764


## Validação - supervisor Naive Bayes

In [54]:
acc_stack = metrics_sk.accuracy_score(y_valid, nb_.predict(X_stack))
f1_stack = metrics_sk.f1_score(y_valid, nb_.predict(X_stack))
precision_stack = metrics_sk.precision_score(y_valid, nb_.predict(X_stack))
recall_stack = metrics_sk.recall_score(y_valid, nb_.predict(X_stack))
auc_stack = metrics_sk.roc_auc_score(y_valid, nb_.predict_proba(X_stack)[:,1])

print('accuracy_stack: ',acc_stack)
print('precision_stack: ',precision_stack)
print('recall_stack: ',recall_stack)
print('F1_stack: ',f1_stack)
print('AUC_stack: ',auc_stack)

accuracy_stack:  0.89338
precision_stack:  0.3266123648017092
recall_stack:  0.3032858028518289
F1_stack:  0.31451716600231455
AUC_stack:  0.7823408105493547


In [55]:
pred_nn_final = NN_Model.predict(X_test_final)
pred_lgbm_final = class_tree.predict_proba(X_test_final)[:,1].reshape(-1, 1)

In [56]:
X_stack_final = np.zeros((pred_nn_final.shape[0], 2))

In [57]:
X_stack_final[:,0] = pred_nn_final.reshape(1, -1)
X_stack_final[:,1] = pred_lgbm_final.reshape(1, -1)
#X_stack_final[:,2] = pred_nn_final.reshape(1, -1)**2
#X_stack_final[:,3] = pred_lgbm_final.reshape(1, -1)**2
#X_stack_final[:,4] = pred_nn_final.reshape(1, -1)**3
#X_stack_final[:,5] = pred_lgbm_final.reshape(1, -1)**3

## Teste final - supervisor Naive Bayes

In [58]:
acc_stack = metrics_sk.accuracy_score(y_test_final, nb_.predict(X_stack_final))
f1_stack = metrics_sk.f1_score(y_test_final, nb_.predict(X_stack_final))
precision_stack = metrics_sk.precision_score(y_test_final, nb_.predict(X_stack_final))
recall_stack = metrics_sk.recall_score(y_test_final, nb_.predict(X_stack_final))
auc_stack = metrics_sk.roc_auc_score(y_test_final, nb_.predict_proba(X_stack_final)[:,1])

print('accuracy_stack: ',acc_stack)
print('precision_stack: ',precision_stack)
print('recall_stack: ',recall_stack)
print('F1_stack: ',f1_stack)
print('AUC_stack: ',auc_stack)

accuracy_stack:  0.8897616828651311
precision_stack:  0.299645390070922
recall_stack:  0.2807308970099668
F1_stack:  0.28987993138936535
AUC_stack:  0.7838798062520407


## Teste final - supervisor logístico

In [59]:
acc_stack = metrics_sk.accuracy_score(y_test_final, lr_.predict(X_stack_final))
f1_stack = metrics_sk.f1_score(y_test_final, lr_.predict(X_stack_final))
precision_stack = metrics_sk.precision_score(y_test_final, lr_.predict(X_stack_final))
recall_stack = metrics_sk.recall_score(y_test_final, lr_.predict(X_stack_final))
auc_stack = metrics_sk.roc_auc_score(y_test_final, lr_.predict_proba(X_stack_final)[:,1])

print('accuracy_stack: ',acc_stack)
print('precision_stack: ',precision_stack)
print('recall_stack: ',recall_stack)
print('F1_stack: ',f1_stack)
print('AUC_stack: ',auc_stack)

accuracy_stack:  0.9187857808547464
precision_stack:  0.43548387096774194
recall_stack:  0.044850498338870434
F1_stack:  0.08132530120481928
AUC_stack:  0.7853238277003032


# Optimizing F1 Score

In [60]:
#### optimizing f1 score
X_opt = np.zeros((X_stack.shape[0], 4))

In [61]:
X_opt[:,0] = 1
X_opt[:,1:3]=X_stack
X_opt[:,3] = y_valid

In [ ]:
#def sigmoid(x): resulta em overflow essa merda
#    return 1/(1+np.exp(-x))

In [62]:
def sigmoid(x): ##corrigido
    if x.all()>=0:
        z = np.exp(-x)
        return 1/(1+z)
    else:
        z = np.exp(-x)
        return z/(1+z)

In [63]:
def sigmoid1(x): ##derivada
    return sigmoid(x)*(1-sigmoid(x))

In [64]:
alpha = 0.25 ##obtive via grid search 
gamma = 2

In [65]:
def f(theta, alpha = alpha,X_opt = X_opt, gamma = gamma):
    A = np.sum(sigmoid(gamma*np.dot(X_opt[X_opt[:,3]==1,:3], theta)))
    npos = X_opt[:,3].sum()
    mpos = np.sum(sigmoid(gamma*np.dot(X_opt[:,:3], theta)))
    F = A/(alpha*npos +(1-alpha)*mpos)
    return -F

In [66]:
def logf(theta, alpha = alpha,X_opt = X_opt, gamma = gamma):
    A = np.sum(sigmoid(gamma*np.dot(X_opt[X_opt[:,3]==1,:3], theta)))
    npos = X_opt[:,3].sum()
    mpos = np.sum(sigmoid(gamma*np.dot(X_opt[:,:3], theta)))
    logF = np.log(A) - np.log((alpha*npos +(1-alpha)*mpos))
    return -logF.round(3)

In [67]:
def df(theta, alpha = alpha,X_opt = X_opt, gamma = gamma):
    A = np.sum(sigmoid(gamma*np.dot(X_opt[X_opt[:,3]==1,:3], theta)))
    npos = X_opt[:,3].sum()
    mpos = np.sum(sigmoid(gamma*np.dot(X_opt[:,:3], theta)))
    h = 1/(alpha*npos + (1-alpha)*mpos)
    dF=np.zeros((theta.shape))
    dA = np.zeros((theta.shape))
    dmpos = np.zeros((theta.shape))
    
    for j in range(X_opt.shape[1]-1):
        dA[j] = np.sum(sigmoid1(gamma*np.dot(X_opt[X_opt[:,3]==1,:3], theta))*gamma*X_opt[X_opt[:,3]==1,j])
        dmpos[j] = np.sum(sigmoid1(gamma*np.dot(X_opt[:,:3], theta))*gamma*X_opt[:,j])
        dF[j] = h*dA[j]-(h**2)*A*(1-alpha)*dmpos[j]
    return -dF
        
    

In [68]:
def dlogf(theta, alpha = alpha,X_opt = X_opt, gamma = gamma):
    A = np.sum(sigmoid(gamma*np.dot(X_opt[X_opt[:,3]==1,:3], theta)))
    npos = X_opt[:,3].sum()
    mpos = np.sum(sigmoid(gamma*np.dot(X_opt[:,:3], theta)))
    dlogF=np.zeros((theta.shape))
    dA = np.zeros((theta.shape))
    dmpos = np.zeros((theta.shape))
    
    for j in range(X_opt.shape[1]-1):
        dA[j] = np.sum(sigmoid1(gamma*np.dot(X_opt[X_opt[:,3]==1,:3], theta))*gamma*X_opt[X_opt[:,3]==1,j])
        dmpos[j] = np.sum(sigmoid1(gamma*np.dot(X_opt[:,:3], theta))*gamma*X_opt[:,j])
        dlogF[j] = (1/A)*dA[j] - (1/(alpha*npos + (1-alpha)*mpos))*(1-alpha)*dmpos[j]
    return -dlogF.round(3)

In [69]:
from scipy import optimize

In [70]:
def predict_opt(X, theta_opt):
    score = sigmoid(np.dot(X[:,:-1], theta_opt))
    predict = (score>0.5).astype(int)
    return (score, predict)

In [140]:
##GRID SEARCH + RANDOM INITIALIZATION --> nao rodar sempre
np.random.seed(42)
#theta_matrix = np.random.normal(loc=0.0, scale=20.0, size=(150, 3)).round(3)
theta_matrix = np.random.randint(low=-30, high=30, size=(200, 3))

list_alpha = [0.25, 0.5, 0.75] 
list_gamma = [2, 4, 8]

lista_alpha = []
lista_gamma = []
lista_theta = []
lista_theta_opt = []
lista_auc = []
lista_precision = []
lista_recall = []
lista_f1 = []

for i in range(len(list_alpha)):
    for j in range(len(list_gamma)):
        for k in range(theta_matrix.shape[0]):
            
            alpha = list_alpha[i]
            gamma = list_gamma[j]
            theta = theta_matrix[k,:].T
            theta_opt = optimize.minimize(logf, theta, jac = dlogf, method = 'bfgs').x
            #theta_opt,k = bfgs_method(f, df, theta, maxiter = None, epsi = 0.0001)
            score, predicted = predict_opt(X_opt, theta_opt)
            print('alpha = ', alpha, 'gamma = ', gamma, 'theta_init = ', theta,
                  'theta_opt = ', theta_opt,
                  'AUC = ', metrics_sk.roc_auc_score(y_valid, score),
                  'precision = ', metrics_sk.precision_score(y_valid, predicted),
                  'recall = ', metrics_sk.recall_score(y_valid, predicted),
                  'F1 = ', metrics_sk.f1_score(y_valid, predicted))
            lista_alpha.append(alpha)
            lista_gamma.append(gamma)
            lista_theta.append(theta)
            lista_theta_opt.append(theta_opt)
            lista_auc.append(metrics_sk.roc_auc_score(y_valid, score))
            lista_precision.append(metrics_sk.precision_score(y_valid, predicted))
            lista_recall.append(metrics_sk.recall_score(y_valid, predicted))
            lista_f1.append(metrics_sk.f1_score(y_valid, predicted))

alpha =  0.25 gamma =  2 theta_init =  [ 8 21 -2] theta_opt =  [ 8. 21. -2.] AUC =  0.7728916603286285 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.25 gamma =  2 theta_init =  [-16  12 -23] theta_opt =  [-260.85558046  256.25168546  237.98941836] AUC =  0.7812008490998671 precision =  0.3429547395388557 recall =  0.24897706137631742 F1 =  0.28850574712643684
alpha =  0.25 gamma =  2 theta_init =  [-10   8  27] theta_opt =  [-11.69222764   6.83343453  26.74908394] AUC =  0.785965436711313 precision =  0.338865836791148 recall =  0.27340359578425294 F1 =  0.30263519077683226
alpha =  0.25 gamma =  2 theta_init =  [-12  -8 -20] theta_opt =  [ 19.3345497   -4.05338032 -19.25455661] AUC =  0.21364899275347876 precision =  0.08064080640806408 recall =  0.9998760074395536 F1 =  0.14924489191590168
alpha =  0.25 gamma =  2 theta_init =  [-20  -7  22] theta_opt =  [-8.48470663  1.43889145 30.69991719] AUC =  0.7872147735311658 precision =  0.3399938987187309 recall = 

alpha =  0.25 gamma =  2 theta_init =  [ -6 -24 -22] theta_opt =  [ 37.68001618 -21.37379378 -21.31609189] AUC =  0.21836632670370665 precision =  0.08064080640806408 recall =  0.9998760074395536 F1 =  0.14924489191590168
alpha =  0.25 gamma =  2 theta_init =  [ -7 -30  13] theta_opt =  [ 33.63794314 -28.14974741  24.67335019] AUC =  0.26701205678787787 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.25 gamma =  2 theta_init =  [-23  -7 -20] theta_opt =  [ 62.23707965   3.14321248 -18.59358819] AUC =  0.5 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.25 gamma =  2 theta_init =  [ 20 -14 -23] theta_opt =  [ 25.55631041  -9.43160466 -21.00034031] AUC =  0.21554423930409067 precision =  0.08063241897256918 recall =  0.9997520148791073 F1 =  0.14922914623086747
alpha =  0.25 gamma =  2 theta_init =  [4 4 2] theta_opt =  [4. 4. 2.] AUC =  0.7792641847047451 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.25 gamma =  2

alpha =  0.25 gamma =  2 theta_init =  [-19   8 -29] theta_opt =  [-155.83910967  190.17488829   98.17151583] AUC =  0.7791330501134744 precision =  0.2813565744600227 recall =  0.36825790452572843 F1 =  0.3189946834219429
alpha =  0.25 gamma =  2 theta_init =  [-28  18   6] theta_opt =  [-21.35980635  24.22582465  10.21735051] AUC =  0.778735667680247 precision =  0.3264007449945677 recall =  0.2607563546187229 F1 =  0.2899090157154673
alpha =  0.25 gamma =  2 theta_init =  [ 18  25 -14] theta_opt =  [ 18.  25. -14.] AUC =  0.7573458666499698 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.25 gamma =  2 theta_init =  [ 28  18 -29] theta_opt =  [ 28.  18. -29.] AUC =  0.6203343894651032 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.25 gamma =  2 theta_init =  [-29  -3  23] theta_opt =  [-15.63522767   9.58961553  33.61831161] AUC =  0.7857160111808422 precision =  0.3436754176610978 recall =  0.26782393056416615 F1 =  0.3010452961672473

alpha =  0.25 gamma =  2 theta_init =  [-6  4 -6] theta_opt =  [-37.5018224   39.12913287  27.1400934 ] AUC =  0.7803543940297721 precision =  0.33934592043155765 recall =  0.2495970241785493 F1 =  0.2876330642280489
alpha =  0.25 gamma =  2 theta_init =  [ -2 -13  15] theta_opt =  [ 6.33993743 -8.84105785 18.35457796] AUC =  0.4578928217262857 precision =  0.08065613122624525 recall =  0.9998760074395536 F1 =  0.14927113702623906
alpha =  0.25 gamma =  2 theta_init =  [-13 -29  23] theta_opt =  [ 19.09608641 -26.55869047  36.10180993] AUC =  0.33563161740833425 precision =  0.08065903725341123 recall =  0.9997520148791073 F1 =  0.14927473178498366
alpha =  0.25 gamma =  2 theta_init =  [  4 -15  10] theta_opt =  [ 10.55812051 -12.49747501  10.64858055] AUC =  0.2645370359412198 precision =  0.0806512911807653 recall =  0.9998760074395536 F1 =  0.14926284809950857
alpha =  0.25 gamma =  2 theta_init =  [  5   2 -27] theta_opt =  [ 14.04163844   8.80108604 -24.74883837] AUC =  0.4398525

alpha =  0.25 gamma =  2 theta_init =  [ 14 -11 -30] theta_opt =  [ 26.61254417  -1.89205415 -27.29048202] AUC =  0.21273923316599697 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.25 gamma =  2 theta_init =  [-23  15 -15] theta_opt =  [-67.12469394  57.28544874  83.19017883] AUC =  0.7830145290863774 precision =  0.3566830745929572 recall =  0.23360198388096715 F1 =  0.2823106316026074
alpha =  0.25 gamma =  2 theta_init =  [-17 -19  20] theta_opt =  [-10.62979454   1.66364144  39.28623853] AUC =  0.7871870645285619 precision =  0.33706690561529273 recall =  0.27985120892746435 F1 =  0.3058058397127566
alpha =  0.25 gamma =  2 theta_init =  [ -8 -16  -3] theta_opt =  [ 16.70077842 -14.32526919  -2.37863012] AUC =  0.22347272161984305 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.25 gamma =  2 theta_init =  [  3 -29   1] theta_opt =  [ 32.49302331 -24.97826464   2.19584318] AUC =  0.22697509032146923 precision =  0.08065 recall =  1.0 

...

alpha =  0.75 gamma =  8 theta_init =  [-16  16  20] theta_opt =  [-16.80172403  15.37127331  19.89779652] AUC =  0.7826031835816505 precision =  0.34076587493940863 recall =  0.2615003099814011 F1 =  0.2959169355970254
alpha =  0.75 gamma =  8 theta_init =  [13 24 21] theta_opt =  [13. 24. 21.] AUC =  0.7797733013813264 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.75 gamma =  8 theta_init =  [ 26 -28   6] theta_opt =  [ 26. -28.   6.] AUC =  0.22964059319653962 precision =  0.08065887247597235 recall =  1.0 F1 =  0.14927721324522925
alpha =  0.75 gamma =  8 theta_init =  [ 20 -24 -10] theta_opt =  [ 25.72331556 -19.74134007  -8.69979707] AUC =  0.22112953533877325 precision =  0.08063241897256918 recall =  0.9997520148791073 F1 =  0.14922914623086747
alpha =  0.75 gamma =  8 theta_init =  [-22   8 -13] theta_opt =  [-186.4838539   156.42695132  279.61487113] AUC =  0.7834719683719504 precision =  0.3365846338535414 recall =  0.2781153130812151 F1 =  0.304569

alpha =  0.75 gamma =  8 theta_init =  [ 24 -17   8] theta_opt =  [ 24. -17.   8.] AUC =  0.23826185182791246 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.75 gamma =  8 theta_init =  [ -4 -22 -16] theta_opt =  [ 30.72762419 -19.13523901 -15.36078667] AUC =  0.21913226786857246 precision =  0.08064161283225664 recall =  0.9998760074395536 F1 =  0.1492462730074123
alpha =  0.75 gamma =  8 theta_init =  [-16  -5  11] theta_opt =  [-5.77982649  1.46577477 18.2686689 ] AUC =  0.7872457504292824 precision =  0.3512674599068805 recall =  0.25257284562926224 F1 =  0.29385458742065784
alpha =  0.75 gamma =  8 theta_init =  [ 29 -18  20] theta_opt =  [ 29. -18.  20.] AUC =  0.2961864670890182 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.75 gamma =  8 theta_init =  [ 1  8 18] theta_opt =  [ 1.  8. 18.] AUC =  0.7844892933769381 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.75 gamma =  8 theta_init =  [ 21   1 -27] the

alpha =  0.75 gamma =  8 theta_init =  [  2 -22  29] theta_opt =  [  6.92999406 -12.46874479  35.58753503] AUC =  0.564970742860557 precision =  0.08063967335828513 recall =  0.9991320520768754 F1 =  0.14923465844375922
alpha =  0.75 gamma =  8 theta_init =  [ 28 -25 -15] theta_opt =  [ 31.35434008 -22.06191083 -13.59829389] AUC =  0.2200496685051782 precision =  0.08062403120156007 recall =  0.9996280223186609 F1 =  0.14921339996298352
alpha =  0.75 gamma =  8 theta_init =  [ -2 -28 -11] theta_opt =  [ 34.28862791 -25.46433987 -10.21014085] AUC =  0.22141260009742322 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.75 gamma =  8 theta_init =  [29 28  5] theta_opt =  [29. 28.  5.] AUC =  0.6420105170804017 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.75 gamma =  8 theta_init =  [-12  -5 -28] theta_opt =  [ 20.96106187   0.29622842 -27.11857351] AUC =  0.21400291001847008 precision =  0.08064322572902916 recall =  0.9998760074395536 F1 =

alpha =  0.75 gamma =  8 theta_init =  [-1  2 -3] theta_opt =  [-39.65262202  42.7390835   22.56651963] AUC =  0.779439137823156 precision =  0.3422459893048128 recall =  0.23806571605703658 F1 =  0.28080438756855575
alpha =  0.75 gamma =  8 theta_init =  [ 16   2 -26] theta_opt =  [ 16.   2. -26.] AUC =  0.22218934972891674 precision =  0.08035857211461502 recall =  0.9959082455052697 F1 =  0.14871733153114788
alpha =  0.75 gamma =  8 theta_init =  [ 29  17 -12] theta_opt =  [ 29.  17. -12.] AUC =  0.7376258192877383 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.75 gamma =  8 theta_init =  [-27   4  18] theta_opt =  [-16.87029873  13.68703446  26.08122011] AUC =  0.783954309345018 precision =  0.339987425337944 recall =  0.26819590824550527 F1 =  0.29985443959243085
alpha =  0.75 gamma =  8 theta_init =  [-14  13  -3] theta_opt =  [-11.17502547  15.66547859  -1.91406253] AUC =  0.7723782257950583 precision =  0.27582568323309065 recall =  0.3241165530068196 F

alpha =  0.75 gamma =  8 theta_init =  [ 4 24  9] theta_opt =  [ 4. 24.  9.] AUC =  0.7783945016815063 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.75 gamma =  8 theta_init =  [ 21 -15 -18] theta_opt =  [ 24.38139831 -12.07645477 -16.60530528] AUC =  0.2171802519172502 precision =  0.08063241897256918 recall =  0.9997520148791073 F1 =  0.14922914623086747
alpha =  0.75 gamma =  8 theta_init =  [19 29 11] theta_opt =  [19. 29. 11.] AUC =  0.7614198709289168 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.75 gamma =  8 theta_init =  [ -1 -12 -14] theta_opt =  [ 18.13196061  -8.16417119 -13.29962977] AUC =  0.2165791277301738 precision =  0.08063241897256918 recall =  0.9997520148791073 F1 =  0.14922914623086747
alpha =  0.75 gamma =  8 theta_init =  [ 25 -12  -3] theta_opt =  [ 25. -12.  -3.] AUC =  0.22264358922823146 precision =  0.08065 recall =  1.0 F1 =  0.14926201822976914
alpha =  0.75 gamma =  8 theta_init =  [27 24 -5] theta_opt

In [141]:
results = pd.DataFrame()
results['alpha']=lista_alpha
results['gamma']=lista_gamma
results['theta']=lista_theta
results['theta_opt']=lista_theta_opt
results['AUC']=lista_auc
results['Precision'] = lista_precision
results['Recall'] = lista_recall
results['F1'] = lista_f1
results.to_csv("resultados_otimizacao_log_table_6.csv")

In [ ]:
#alpha =  0.25 gamma =  2 theta_init =  [-1 -4  9] theta_opt =  [-568.50174834  322.85256065 2247.08499949] AUC =  0.7829109328873998 precision =  0.2642267969778887 recall =  0.4726596404215747 F1 =  0.338964965321003
#alpha =  0.25 gamma =  2 theta_init =  [0 3 5] theta_opt =  [-431.15928105  220.73593698 1731.39766752] AUC =  0.7840137316349044 precision =  0.26989136817381093 recall =  0.45592064476131433 F1 =  0.3390658859329614
#0.1	2	[-12.902   7.228  30.761]	[-12.902   7.228  30.761]	0.7882440904853698	0.34440142942227514	0.28679479231246124	0,312969353
#0.1	2	[-21.154  16.451 -24.417]	[-89.98832355  91.3083205   81.7098475 ]	0.7784056931783964	0.3293559516963772	0.28406695598264103	0,305039611
#0.25	2	[ -5.051 -24.956  32.648]	[-11.94642079   0.16685741  50.35630243]	0.7865348206371445	0.33280689160086147	0.2874147551146931	0,308449767
#0.25	2	[-18.16  -28.246  29.313]	[-18.25223462   5.61979402  59.03788049]	0.7871950151039016	0.3361270401411557	0.28344699318040917	0,307547424

#0.25	2	[ -0.717  31.293 -52.395]	[-1968.81416686  2356.06814755  1390.76945771]	0.7442478049348258	0.2828002298410266	0.3661500309981401	0,319122494

#	0.25	2	[ -9.685  25.338 -14.153]	[-233.99079602  188.47355835  372.24989718]	0.7836955872384971	0.33852496564360973	0.2748915065096094	0,303407691

#0.25	2	[-19   8 -29]	[-155.83910967  190.17488829   98.17151583]	0.7791330501134744	0.2813565744600227	0.36825790452572843	0,318994683
#0.25	2	[-20  -7  22]	[-8.48470663  1.43889145 30.69991719]	0.7872147735311658	0.3399938987187309	0.2763794172349659	0,304903905
#0.25	2	[-22  -5  22]	[-10.98538955   5.16173794  30.62632766]	0.786705006107748	0.33062937062937064	0.2931184128952263	0,310745974


In [173]:
alpha = 0.25
gamma = 2
theta_opt = np.array([-155.83910967,  190.17488829,   98.17151583])

In [174]:
score, predicted = predict_opt(X_opt, theta_opt)

### Performance em validação (usado para otimizar)

In [175]:
print('AUC:', metrics_sk.roc_auc_score(y_valid, score))
print('PRECISION: ', metrics_sk.precision_score(y_valid, predicted))
print('RECALL: ', metrics_sk.recall_score(y_valid, predicted))
print('F1: ', metrics_sk.f1_score(y_valid, predicted))

AUC: 0.7791330501134744
PRECISION:  0.2813565744600227
RECALL:  0.36825790452572843
F1:  0.3189946834219429


In [176]:
X_opt_final = np.zeros((X_stack_final.shape[0], 4))
X_opt_final[:,0] = 1
X_opt_final[:,1:3]=X_stack_final
X_opt_final[:,3] = y_test_final

In [177]:
score_final, predicted_final = predict_opt(X_opt_final, theta_opt)

In [178]:
X_opt_final.shape

(7511, 4)

In [179]:
X_stack_final.shape

(7511, 2)

### Performance no teste final

In [180]:
print('ACCURACY', metrics_sk.roc_auc_score(y_test_final, predicted_final))
print('AUC', metrics_sk.roc_auc_score(y_test_final, score_final))
print('PRECISION', metrics_sk.precision_score(y_test_final, predicted_final))
print('RECALL', metrics_sk.recall_score(y_test_final, predicted_final))
print('F1', metrics_sk.f1_score(y_test_final, predicted_final))

ACCURACY 0.6434209507652641
AUC 0.783824266965569
PRECISION 0.27552275522755226
RECALL 0.37209302325581395
F1 0.31660777385159006


In [ ]:
from eli5.permutation_importance import get_score_importances

In [ ]:
def score(X, y):
    y_pred = NN_Model.predict(X)
    return metrics_sk.roc_auc_score(y, y_pred)

base_score, score_decreases = get_score_importances(score, X_valid, y_valid)
feature_importances = np.mean(score_decreases, axis=0)

In [ ]:
feature_importance_dict = {}
for i, feature_name in enumerate(feats):
    feature_importance_dict[feature_name]=feature_importances[i]
print(dict(sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)[:-1]))

In [ ]:
dict(sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)[:-1])

In [ ]:
feature_importance_dict = {}
for i, feature_name in enumerate(feats):
    feature_importance_dict[feature_name]=class_tree.feature_importances_[i]
print(dict(sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)[:-1]))

In [ ]:
dict(sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)[:-1])

In [ ]:
def score(X, y):
    y_pred = class_tree.predict(X)
    return metrics_sk.roc_auc_score(y, y_pred)

base_score, score_decreases = get_score_importances(score, X_valid, y_valid)
feature_importances = np.mean(score_decreases, axis=0)

In [ ]:
feature_importance_dict = {}
for i, feature_name in enumerate(feats):
    feature_importance_dict[feature_name]=feature_importances[i]
print(dict(sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)[:-1]))

In [ ]:
dict(sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)[:-1])

In [ ]:
feat_selected = ["mean_knn_target",
"EXT_SOURCE_3",
"EXT_SOURCE_2",
"EXT_SOURCE_1",
"SUM(previous.CNT_PAYMENT)",
"amt_credit_sum_consumer_credit_bureau",
"avg_pos_cash_cnt_installment_future_previous",
"CODE_GENDER_M",
"amt_credit_sum_debt_credit_card_bureau",
"cnt_refused_previous",
"NAME_CONTRACT_TYPE_Revolving loans",
"max_pos_cash_sk_dpd_def_previous",
"SUM(previous.DAYS_FIRST_DRAWING)",
"amt_paid_down",
"NAME_EDUCATION_TYPE_Higher education",
"STD(previous.DAYS_FIRST_DRAWING)",
"NAME_FAMILY_STATUS_Married",
"MIN(previous.CNT_PAYMENT)",
"cnt_yield_low_normal_previous",
"avg_amt_application_previous",
"credit_income_ratio",
"remaining_pmts",
"OCCUPATION_TYPE_Sales staff",
"MAX(previous.NFLAG_LAST_APPL_IN_DAY)",
"FLAG_EMAIL",
"missing_rate",
"REGION_RATING_CLIENT_W_CITY_2",
"COUNT(bureau)",
"gap_avg_application_credit_previous",
"days_phone_change_over_gap_amt_installment",
"FLAG_OWN_CAR_Y",
"avg_pos_cash_cnt_installment_previous",
"NAME_EDUCATION_TYPE_Secondary / secondary special",
"YEARS_BUILD_MODE",
"UI_credit_card_bureau",
"ENTRANCES_AVG",
"num_loans_l1m",
"STD(bureau.CREDIT_DAY_OVERDUE)",
"STD(previous.AMT_GOODS_PRICE)",
"max_interest_rate_privileged_previous",
"cnt_contracts_bureau",
"SKEW(previous.DAYS_TERMINATION)",
"MEAN(previous.DAYS_LAST_DUE_1ST_VERSION)",
"MIN(previous.AMT_APPLICATION)",
"MIN(bureau.DAYS_CREDIT)",
"delinquency_6_mths_consumer_credit_bureau",
"MAX(previous.DAYS_FIRST_DRAWING)",
"NAME_EDUCATION_TYPE_Lower secondary",
"MIN(previous.HOUR_APPR_PROCESS_START)",
"avg_amt_credit_card_receivable_principal_previous",
"MEAN(bureau.AMT_CREDIT_SUM)",
"amt_credit_max_overdue_bureau",
"DAYS_ID_PUBLISH",
"tot_debt_lim_ratio",
"avg_amt_credit_card_total_receivable_previous",
"max_interest_rate_primary_previous",
"MEDIAN(previous.NFLAG_INSURED_ON_APPROVAL)",
"num_loans_l6m",
"STD(bureau.DAYS_CREDIT)",
"MEAN(previous.RATE_INTEREST_PRIMARY)",
"FLOORSMIN_AVG",
"SUM(bureau.AMT_CREDIT_SUM_DEBT)",
"avg_cnt_credit_card_drawings_atm_previous",
"APARTMENTS_MEDI",
"LIVINGAREA_MODE",
"delinquency_12_24_mths_microloan_bureau",
"SKEW(bureau.DAYS_CREDIT)",
"LANDAREA_AVG",
"num_loans_l3m",
"FLAG_DOCUMENT_13",
"MAX(bureau.AMT_CREDIT_SUM_DEBT)",
"MAX(previous.CNT_PAYMENT)",
"delinquency_6_12_mths_credit_card_bureau",
"MAX(bureau.AMT_CREDIT_SUM_LIMIT)",
"MEAN(previous.AMT_CREDIT)",
"MAX(bureau.DAYS_CREDIT_UPDATE)",
"WALLSMATERIAL_MODE_Monolithic",
"avg_amt_credit_card_drawings_atm_previous",
"MEDIAN(previous.CNT_PAYMENT)",
"avg_cnt_credit_card_instalment_mature_cum_previous",
"FLAG_PHONE",
"YEARS_BEGINEXPLUATATION_AVG",
"SUM(previous.DAYS_TERMINATION)",
"income_over_credit",
"days_phone_change_times_gap_days_installment",
"SUM(previous.AMT_GOODS_PRICE)",
"LANDAREA_MEDI",
"max_cnt_payment_previous",
"SUM(bureau.AMT_CREDIT_SUM_OVERDUE)",
"MEAN(previous.NFLAG_LAST_APPL_IN_DAY)",
"MIN(previous.RATE_DOWN_PAYMENT)",
"SKEW(bureau.AMT_CREDIT_SUM_LIMIT)",
"SUM(previous.RATE_DOWN_PAYMENT)",
"cnt_portfolio_cash_previous",
"MEDIAN(bureau.CNT_CREDIT_PROLONG)",
"SKEW(previous.DAYS_DECISION)",
"NAME_INCOME_TYPE_State servant",
"STD(previous.AMT_CREDIT)",
"MEAN(bureau.DAYS_ENDDATE_FACT)",
"NAME_TYPE_SUITE_Other_A",
"FLOORSMIN_MEDI",
"DEF_30_CNT_SOCIAL_CIRCLE",
"REG_REGION_NOT_WORK_REGION",
"SKEW(bureau.AMT_CREDIT_MAX_OVERDUE)",
"NONLIVINGAREA_MEDI",
"delinquency_6_12_mths_microloan_bureau",
"avg_amt_credit_card_recivable_previous",
"delinquency_long_total_bureau",
"FLAG_WORK_PHONE",
"cnt_channel_contact_center_previous",
"SUM(previous.DAYS_FIRST_DUE)",
"OWN_CAR_AGE",
"AMT_GOODS_PRICE",
"cnt_appl_previous",
"delinquency_12_24_mths_consumer_credit_bureau",
"cnt_cash_previous",
"NAME_TYPE_SUITE_Family",
"MEAN(previous.AMT_DOWN_PAYMENT)",
"FLAG_DOCUMENT_3",
"NAME_TYPE_SUITE_Group of people",
"cnt_seller_furniture_previous",
"delinquency_6_12_mths_total_bureau",
"CODE_GENDER_XNA",
"amt_credit_sum_working_capital_bureau",
"MEDIAN(previous.RATE_DOWN_PAYMENT)",
"DAYS_REGISTRATION",
"DEF_60_CNT_SOCIAL_CIRCLE",
"SUM(bureau.DAYS_CREDIT_ENDDATE)",
"MIN(bureau.DAYS_CREDIT_ENDDATE)",
"delinquency_24_36_mths_consumer_credit_bureau",
"MAX(previous.AMT_DOWN_PAYMENT)",
"UI_credit_sum_total_bureau",
"SKEW(bureau.DAYS_ENDDATE_FACT)",
"SKEW(previous.AMT_DOWN_PAYMENT)",
"NAME_INCOME_TYPE_Student",
"MEDIAN(bureau.DAYS_CREDIT_UPDATE)",
"credit_income_cluster_4",
"YEARS_BEGINEXPLUATATION_MEDI",
"NAME_HOUSING_TYPE_Office apartment",
"max_credit_card_sk_dpd_def_previous",
"SKEW(previous.AMT_APPLICATION)",
"cnt_product_xsell_previous",
"WALLSMATERIAL_MODE_Stone, brick",
"cnt_yield_low_action_previous",
"MAX(previous.DAYS_DECISION)",
"amt_credit_sum_overdue_microloan_bureau",
"MEAN(previous.DAYS_TERMINATION)",
"OBS_30_CNT_SOCIAL_CIRCLE",
"cnt_seller_electronics_previous",
"MAX(previous.HOUR_APPR_PROCESS_START)",
"WALLSMATERIAL_MODE_Panel",
"MEDIAN(previous.AMT_APPLICATION)",
"down_payment_times_cnt_refused_previous",
"MEDIAN(previous.AMT_CREDIT)",
"max_installments_gap_amt_payment_previous",
"STD(previous.CNT_PAYMENT)",
"FLOORSMIN_MODE",
"FLOORSMAX_AVG",
"STD(bureau.DAYS_ENDDATE_FACT)"]

In [ ]:
data[feat_selected].head()

In [ ]:
data.head()